In [15]:
import pandas as pd
from gcs_utils import get_file_info, upload_to_gcs, delete_file_from_gcs, rename_file_in_gcs, download_file_from_gcs, GCS_BUCKET_NAME
import os
import dotenv

dotenv.load_dotenv()


True

In [ ]:
# Get file information
files = get_file_info(GCS_BUCKET_NAME)

# Convert file information to DataFrame
df = pd.DataFrame(files)

In [14]:
print(df)

                                                name      size pages
0                                 99745 - ACCENTURE/         0   N/A
1  99745 - ACCENTURE/1. LATAM One Pack - Contrato...  11097221   302
2             Oferta Comercial Citrix LATAM 2023.pdf   2266218     9


In [24]:
# Filtrar los nombres que no terminan con "/"
df_filtered = df[~df['name'].str.endswith('/')]

# Obtener solo los nombres como una lista
nombres_archivos = df_filtered['name'].tolist()

print("Lista de nombres de archivos:")
print(nombres_archivos)


Lista de nombres de archivos:
['99745 - ACCENTURE/1. LATAM One Pack - Contrato Firmado Completo.pdf', 'Oferta Comercial Citrix LATAM 2023.pdf']


In [25]:
from langchain_google_community import GCSFileLoader

from langchain_community.document_loaders import PyPDFLoader


def load_pdf(file_path):
    return PyPDFLoader(file_path)


loader = GCSFileLoader(
    project_name=os.getenv("GCP_PROJECT_NAME"), bucket=os.getenv("GCS_BUCKET_NAME") , blob="99745 - ACCENTURE/1. LATAM One Pack - Contrato Firmado Completo.pdf", loader_func=load_pdf)

print(loader)


In [26]:
loader.load()

[Document(metadata={'source': 'gs://contratos_rag/99745 - ACCENTURE/1. LATAM One Pack - Contrato Firmado Completo.pdf', 'page': 0}, page_content='Master Service Agreement  \n \nLatam / Accenture Confidencial                       Página 1 de 24 \n \nEste documento está sometido a la validación interna de Accenture y Latam, y está sujeto a una revisión e iteración adicional es en \nfunción del alcance confirmado de los servicios, incluida la complementación de cualquier disposición  legal adicional para tener en \ncuenta la especificación de servicios acordada.  \n CONTRATO DE PRESTACIÓN DE SERVICIOS  \n \nEl presente Contrato de Prestación de Servicios o Master Service Agreement (en  adelante “MSA” (por sus \nsiglas en inglés) o “Acuerdo") se celebra el día ________________ entre Accenture  Chile, Asesorías y \nServicios Ltda. (en adelante, “Accenture”), RUT N°78.809.770 -0, con domicilio en Apoquindo 5550 piso 14 \ntorre Apoquindo, comuna de Las Condes, Santiago , ("Accenture "), y LA

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Cargar el documento
documento = loader.load()

# Crear el divisor de texto
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

# Dividir el documento en chunks
chunks = text_splitter.split_documents(documento)

print(f"El documento se ha dividido en {len(chunks)} chunks.")

# Imprimir los primeros 5 chunks como ejemplo
print("\nPrimeros 5 chunks:")
for i, chunk in enumerate(chunks[:5], 1):
    print(f"\nChunk {i}:")
    print(chunk.page_content[:150] + "...")  # Mostrar los primeros 150 caracteres de cada chunk


El documento se ha dividido en 1093 chunks.

Primeros 5 chunks:

Chunk 1:
Master Service Agreement  
 
Latam / Accenture Confidencial                       Página 1 de 24 
 
Este documento está sometido a la validación inter...

Chunk 2:
89.862.200 - 2, con domicilio en  Avenida Américo Vespucio número 901, comuna de Renca , ( "LATAM "). 
En conjunto denominadas como "las Partes " o, i...

Chunk 3:
vigencia de 58 meses (" Plazo "). La vigencia de los Servicios será definida en el Anexo A 
correspondiente.   
1.2 Definiciones.  Todos los términos ...

Chunk 4:
Anexo B : Servicios de Transición  
Anexo C : Compensación  
Apéndice C.1: Cargo Fijo y proyección de Cargo Base  
Apéndice C.2: Indicadores de Línea ...

Chunk 5:
Master Service Agreement  
 
Latam / Accenture Confidencial                       Página 2 de 24 
 
Este documento está sometido a la validación inter...
